In [110]:
from utils import get_spark_session

In [111]:
spark = get_spark_session("iceberg", "bronze")
print(spark)

In [112]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [113]:
df = spark.read.csv("s3a://landing/marca_carro.csv", header=True, inferSchema=True)
df.show()

+------------+---------+
| marca_carro|cod_marca|
+------------+---------+
|       Acura|        1|
|Aston Martin|        2|
|        Audi|        3|
|      Austin|        4|
|         BMW|        5|
|     Bentley|        6|
|     Bugatti|        7|
|       Buick|        8|
|    Cadillac|        9|
|   Chevrolet|       10|
|    Chrysler|       11|
|     Citroën|       12|
|      Daewoo|       13|
|       Dodge|       14|
|       Eagle|       15|
|     Ferrari|       16|
|        Ford|       17|
|         GMC|       18|
|         Geo|       19|
|       Honda|       20|
+------------+---------+
only showing top 20 rows



In [114]:
spark.sql("""
CREATE NAMESPACE IF NOT EXISTS nessie.bronze
""")


DataFrame[]

In [115]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.bronze.tabela_carro (
    marca_carro STRING,
    cod_marca INT
)
USING iceberg
LOCATION 's3a://bronze/marca_carro/'
""")


DataFrame[]

In [116]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|       nessie|
|spark_catalog|
+-------------+



In [122]:
spark.sql("SHOW TABLES IN nessie.bronze").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|   bronze| marca_carro|      false|
|   bronze|tabela_carro|      false|
+---------+------------+-----------+



In [126]:
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")

spark.conf.set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
spark.conf.set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
spark.conf.set("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)

In [127]:
df.write.format("iceberg").mode("overwrite").save("nessie.bronze.tabela_carro")

24/11/20 20:28:22 ERROR TaskSetManager: Task 0 in stage 4.0 failed 4 times; aborting job
24/11/20 20:28:22 ERROR OverwriteByExpressionExec: Data source write support IcebergBatchWrite(table=nessie.bronze.tabela_carro, format=PARQUET) is aborting.
24/11/20 20:28:22 ERROR OverwriteByExpressionExec: Data source write support IcebergBatchWrite(table=nessie.bronze.tabela_carro, format=PARQUET) aborted.


Py4JJavaError: An error occurred while calling o1869.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4.0 (TID 10) (172.18.0.6 executor 3): software.amazon.awssdk.services.s3.model.S3Exception: The Access Key Id you provided does not exist in our records. (Service: S3, Status Code: 403, Request ID: 1809C7271491E07C, Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleErrorResponse(AwsXmlPredicatedResponseHandler.java:156)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleResponse(AwsXmlPredicatedResponseHandler.java:108)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handle(AwsXmlPredicatedResponseHandler.java:85)
	at software.amazon.awssdk.protocols.xml.in

In [ ]:
spark.sql("SHOW TABLES IN nessie.bronze").show()

In [109]:
spark.stop()